In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [8]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
#indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

# pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
# pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

pairs_train = [x for x in pairs if int(x[0].split('_')[-1]) <= 9]
pairs_test = [x for x in pairs if int(x[0].split('_')[-1]) >= 10]

In [9]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [11]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
position_embeddings = PositionEmbedding(sequence_length=len(x_masked_train[0]))(word_embeddings)
encoder_output = word_embeddings + position_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 6s - loss: 3.2925 - val_loss: 3.2452 - 6s/epoch - 6s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2646 - val_loss: 3.2214 - 191ms/epoch - 191ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.2415 - val_loss: 3.2011 - 182ms/epoch - 182ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.2217 - val_loss: 3.1833 - 226ms/epoch - 226ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.2041 - val_loss: 3.1673 - 175ms/epoch - 175ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1881 - val_loss: 3.1528 - 179ms/epoch - 179ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.1735 - val_loss: 3.1398 - 174ms/epoch - 174ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.1602 - val_loss: 3.1273 - 210ms/epoch - 210ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.1477 - val_loss: 3.1155 - 213ms/epoch - 213ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.1357 - val_loss: 3.1043 - 204ms/epoch - 204ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.1244 - val_loss: 3.0938 - 224ms/epoch - 224ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.1137 - val_loss: 3.0841 - 210ms/epoch - 210ms/step
Epoch 1

Epoch 99/2000
1/1 - 0s - loss: 2.5123 - val_loss: 2.5174 - 185ms/epoch - 185ms/step
Epoch 100/2000
1/1 - 0s - loss: 2.5043 - val_loss: 2.5095 - 190ms/epoch - 190ms/step
Epoch 101/2000
1/1 - 0s - loss: 2.4962 - val_loss: 2.5029 - 201ms/epoch - 201ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.4896 - val_loss: 2.4964 - 181ms/epoch - 181ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.4831 - val_loss: 2.4893 - 184ms/epoch - 184ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.4760 - val_loss: 2.4815 - 178ms/epoch - 178ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.4684 - val_loss: 2.4746 - 180ms/epoch - 180ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.4615 - val_loss: 2.4701 - 182ms/epoch - 182ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.4568 - val_loss: 2.4699 - 180ms/epoch - 180ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.4570 - val_loss: 2.4605 - 181ms/epoch - 181ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.4476 - val_loss: 2.4488 - 183ms/epoch - 183ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.4360 - val_loss: 2.4491 - 167ms/

Epoch 196/2000
1/1 - 0s - loss: 2.1605 - val_loss: 2.1672 - 163ms/epoch - 163ms/step
Epoch 197/2000
1/1 - 0s - loss: 2.1589 - val_loss: 2.1656 - 136ms/epoch - 136ms/step
Epoch 198/2000
1/1 - 0s - loss: 2.1573 - val_loss: 2.1637 - 154ms/epoch - 154ms/step
Epoch 199/2000
1/1 - 0s - loss: 2.1555 - val_loss: 2.1620 - 143ms/epoch - 143ms/step
Epoch 200/2000
1/1 - 0s - loss: 2.1538 - val_loss: 2.1605 - 123ms/epoch - 123ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.1523 - val_loss: 2.1589 - 118ms/epoch - 118ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.1507 - val_loss: 2.1572 - 125ms/epoch - 125ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.1490 - val_loss: 2.1555 - 113ms/epoch - 113ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.1474 - val_loss: 2.1541 - 117ms/epoch - 117ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.1459 - val_loss: 2.1526 - 123ms/epoch - 123ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.1444 - val_loss: 2.1510 - 116ms/epoch - 116ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.1428 - val_loss: 2.1494 - 117ms

Epoch 293/2000
1/1 - 0s - loss: 2.0608 - val_loss: 2.0690 - 128ms/epoch - 128ms/step
Epoch 294/2000
1/1 - 0s - loss: 2.0602 - val_loss: 2.0685 - 118ms/epoch - 118ms/step
Epoch 295/2000
1/1 - 0s - loss: 2.0596 - val_loss: 2.0680 - 114ms/epoch - 114ms/step
Epoch 296/2000
1/1 - 0s - loss: 2.0591 - val_loss: 2.0674 - 130ms/epoch - 130ms/step
Epoch 297/2000
1/1 - 0s - loss: 2.0585 - val_loss: 2.0669 - 139ms/epoch - 139ms/step
Epoch 298/2000
1/1 - 0s - loss: 2.0580 - val_loss: 2.0664 - 113ms/epoch - 113ms/step
Epoch 299/2000
1/1 - 0s - loss: 2.0575 - val_loss: 2.0659 - 130ms/epoch - 130ms/step
Epoch 300/2000
1/1 - 0s - loss: 2.0569 - val_loss: 2.0654 - 103ms/epoch - 103ms/step
Epoch 301/2000
1/1 - 0s - loss: 2.0564 - val_loss: 2.0649 - 109ms/epoch - 109ms/step
Epoch 302/2000
1/1 - 0s - loss: 2.0559 - val_loss: 2.0644 - 111ms/epoch - 111ms/step
Epoch 303/2000
1/1 - 0s - loss: 2.0554 - val_loss: 2.0639 - 144ms/epoch - 144ms/step
Epoch 304/2000
1/1 - 0s - loss: 2.0549 - val_loss: 2.0634 - 113ms

In [12]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [13]:
(np.mean(acc), np.mean(prob))

(0.0, 0.018784668)

In [14]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
position_embeddings = PositionEmbedding(sequence_length=len(x_masked_train[0]))(word_embeddings)
encoder_output = word_embeddings + position_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 6s - loss: 3.5969 - val_loss: 3.2553 - 6s/epoch - 6s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2400 - val_loss: 3.0435 - 442ms/epoch - 442ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.0411 - val_loss: 2.9438 - 432ms/epoch - 432ms/step
Epoch 4/2000
1/1 - 0s - loss: 2.9469 - val_loss: 2.9176 - 431ms/epoch - 431ms/step
Epoch 5/2000
1/1 - 1s - loss: 2.9201 - val_loss: 2.8815 - 513ms/epoch - 513ms/step
Epoch 6/2000
1/1 - 1s - loss: 2.8821 - val_loss: 2.8532 - 506ms/epoch - 506ms/step
Epoch 7/2000
1/1 - 0s - loss: 2.8509 - val_loss: 2.8274 - 471ms/epoch - 471ms/step
Epoch 8/2000
1/1 - 0s - loss: 2.8212 - val_loss: 2.8148 - 495ms/epoch - 495ms/step
Epoch 9/2000
1/1 - 0s - loss: 2.8052 - val_loss: 2.8164 - 466ms/epoch - 466ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.8044 - val_loss: 2.8180 - 451ms/epoch - 451ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.8050 - val_loss: 2.8127 - 457ms/epoch - 457ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.7998 - val_loss: 2.8032 - 483ms/epoch - 483ms/step
Epoch 1

In [15]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [16]:
(np.mean(acc), np.mean(prob))

(0.0, 0.007304905)

In [17]:
x_masked_test

array([[10,  0,  1, 10],
       [10,  0,  2, 10],
       [10,  0,  3, 10],
       ...,
       [19, 18, 15, 19],
       [19, 18, 16, 19],
       [19, 18, 17, 19]])